In [2]:
%pylab inline
from CoolProp.CoolProp import PropsSI

plt.rcParams['figure.figsize'] = (10,6)
plt.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


In [3]:
atm_to_psia = 14.7        # psia / atm
atm_to_Pa = 101325.       # Pa / atm
psi_to_Pa = atm_to_Pa / atm_to_psia

# Conversion to go from RPM, gpm, feet to RPM, m^3/s, m
SI_to_US = (264.172 * 60)**0.5 / (3.28084)**0.75

### References
- [NASA SP-8109](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19740020848.pdf)
- [NASA SP-8121](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19780022641.pdf)
- [NASA SP-125 (Huzel and Huang)](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19710019929.pdf)
- [Centrifugal Pumps, Designs and Applications (Lobanoff)](https://catatanabimanyu.files.wordpress.com/2011/09/centrifugal_pumps_2e.pdf)
- [Rotating and Positive Displacement Pumps for Low-thrust Engines CR-72965](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19740015230.pdf)

### Staring from requirements

In [9]:
g0 = 9.80665         # m / sec^2
N = 60000.
psi = 0.70           # Head coefficient, see Lobanoff, Ross pg. 181
phi = 1.3            # velocity ratio throat to impeller, pg. 182
Ss = 7500.           # in US units of RPM, GPM, ft

mdot = 1.00          #kg / s
rho_lox = PropsSI('D', 'P', 101325, 'Q', 0, 'Oxygen')
Q = mdot / rho_lox
dP = 480 * psi_to_Pa
H = dP / rho_lox

D2 = 60 * np.sqrt(g0 * H / psi) / pi / N
d1 = 2 * (Q / pi / phi)**0.5 * (psi / g0 / H)**0.25

#d1 = 0.1 * .0254      # Outlet throad diameter, m
#D2 = 3. * .0254       # Impeller outer diameter, m
Ds = 10e-3             # shaft diameter, m
d2 = 0.234 * .0254     # Pump outlet diamter, m
b2 = 0.25 * .0254      # Impeller outer height, m
alpha = 15.            # deg
theta = 30.            # deg

omega = N / 60. * 2. * pi
u2 = omega * D2 / 2.
H = psi * u2**2 / g0
Q = phi * u2 * pi * d1**2 / 4.

Ns0 = 4847. * phi**0.5 * d1 / psi**0.75 / D2
Ns = N * Q**0.5 / H**0.75
Ss_si = Ss / SI_to_US
NPSHR = H * (Ns / Ss_si)**1.333
NPSPR = NPSHR * rho_lox * g0
De = 5.1 * (Q * 264.172 * 60 / N)**0.33 * .0254
Ve = Q / (pi * De**2 / 4)
V1 = Q / (pi * d1**2 / 4)
V2 = Q / (pi * d2**2 / 4)
ue = omega * De / 2.
Pe = ue**2 * rho_lox / g0
h1 = d1 * (D2 / De)

print 'Mdot: %.3f kg/s' % (Q*rho_lox)
print 'Head rise: %.1f m' % H
print 'Specific speed : %.1f (%.4f SI)' % (Ns * SI_to_US, Ns * 2 * pi / 60)
print 'NPSHR: %.1f m' % NPSHR
print 'Suction pressure required: %.1f psi' % (NPSPR / psi_to_Pa)
print 'Eye inlet diameter: %.3f in' % (De / .0254)
print 'Eye inlet velocity: %.1f m/s' % Ve
print 'Rotor diameter: %.1f in' % (D2 / .0254)
print 'Blade inner height: %.3f in' % (h1 / .0254)
print 'Throat diameter: %.3f in ' % (d1 / .0254)
print 'Throat velocity: %.1f m/s' % V1
print 'Outlet velocity: %.1f m/s' % V2
print 'Eye tangential velocity: %.1f m/s' % ue
print 'Shaft seal surfaces speed: %.1f m/s' % (Ds * omega / 2)
print 'Seal dynamic pressure ~ %.1f psia' % (Pe / psi_to_Pa)

Mdot: 1.000 kg/s
Head rise: 2899.3 m
Specific speed : 232.2 (0.4707 SI)
NPSHR: 28.2 m
Suction pressure required: 45.8 psi
Eye inlet diameter: 0.322 in
Eye inlet velocity: 16.7 m/s
Rotor diameter: 2.5 in
Blade inner height: 0.637 in
Throat diameter: 0.081 in 
Throat velocity: 262.0 m/s
Outlet velocity: 31.6 m/s
Eye tangential velocity: 25.7 m/s
Shaft seal surfaces speed: 31.4 m/s
Seal dynamic pressure ~ 11.1 psia


### Seal Options
The Lehner 3040 motor has a 10mm shaft.

- [John Crane Type 2100](https://www.johncrane.com/~/media/J/Johncrane_com/Files/Products/Technical%20Specification/Seals/TD-2100-8PG-BW-OCT2015.pdf) - very basic carbon mechanical (face) seal.  Available in 10mm shaft size.  Not rated for low temperature, high speed or oxygen service
- [John Crane Type 285](https://www.johncrane.com/~/media/J/Johncrane_com/Files/Products/Technical%20Specification/Seals/TD-285-4PG-BW-OCT2015.pdf) - Specified for LOX service.  Pressure and speed quite a bit lower than desired (100 psig & 10,000 RPM).  Not clear what shaft sizes available
- [John Crane Type 59U](https://www.johncrane.com/~/media/J/Johncrane_com/Files/Products/Technical%20Specification/Seals/TD-59U-59B-8PG-BW-OCT2015.pdf) - PTFE wedge should be compatible with LOX.  Only goes down to 14mm in catalog, would need to contact to see if they could do 10mm
- [Daybreak Seals](https://www.daybreakworld.com/multispring-mechanical-seals.php) has some interesting options that mention Cryogenic operation and go down to 10mm
- [Parker Catalog](https://www.parker.com/literature/Engineered%20Polymer%20Systems/5350.pdf) - looks like FlexiCase are the right product.  Part number M(metric)-0102(modified PTFE)-CMN(single machined lip)-0600(0.25in gland xsection)-01000(10mm shaft)-060(6mm width)-V(FKM flourocarbon gasket)-A(aluminum): [M-0102-CMN-0600-01000-060-V-A](http://ph.parker.com/us/17061/en/high-speed-low-torque-metal-case-ptfe-lip-rotary-shaft-seal-flexicase-cmn-cme)